# Import thư viện cần thiết


In [166]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import json

# Thu thập top 1000 tên manga phổ biến và link chứa các thông tin chi tiết về manga như rating, vote, ... và lưu vào một dictionary

In [160]:
driver=webdriver.Chrome()

In [4]:
def manga_item(path):
    manga_list=[] #lưu trữ thông tin name và href manga
    ranked=1
    for limit in range(0,1000,50): #bước nhảy là 50 do per_page=50
        _path=path.format(limit)
        driver.get(_path)
        elem = driver.find_elements(By.CLASS_NAME, "manga_h3") #list các element chứa tên manga và href dẫn tới thông tin manga
        for e in elem:
            manga_dict = {} #dùng dictionary để store name and href
            manga_dict['name'] = e.text
            link=e.find_element(By.LINK_TEXT, e.text) 
            manga_dict['href'] = link.get_attribute("href") #lấy link thông tin của element
            manga_dict['ranked']=ranked
            manga_list.append(manga_dict)
            ranked=ranked+1
    return manga_list

In [5]:
path="https://myanimelist.net/topmanga.php?type=manga&limit={}"
manga_list=manga_item(path)

In [6]:
len(manga_list) #kiểm tra có đủ phần tử không

1000

In [168]:
with open('manga_list.json', 'w') as f: #save to .json
    json.dump(data, f)


# Thu thập thông tin cho mỗi manga trong manga list

In [154]:
def manga_info(manga_dict):
    manga={}
    manga['name']=manga_dict['name']
    manga['ranked']=manga_dict['ranked']
    driver.get(manga_dict['href']) #truy cập vào đường dẫn chứa thông tin manga
    manga['score']=driver.find_element(By.CSS_SELECTOR,"div.score-label").text #tìm score
    manga['popularity']=driver.find_element(By.CSS_SELECTOR,"span.numbers.popularity").text.split('#')[1]  #tìm ranked
    manga['favorite']=driver.find_element(By.XPATH, "//div[@class='spaceit_pad'][span[text()='Favorites:']]").text.split(': ')[1]
    manga['published']=driver.find_element(By.XPATH, "//div[@class='spaceit_pad'][span[text()='Published:']]").text.split(' to')[0].split(': ')[1]
    try:
        manga['genre']=driver.find_element(By.XPATH, "//div[@class='spaceit_pad'][span[text()='Genres:']]").text.split(': ')[1]
    except NoSuchElementException:
        try:
            manga['genre']=driver.find_element(By.XPATH, "//div[@class='spaceit_pad'][span[text()='Genre:']]").text.split(': ')[1]
        except NoSuchElementException:
            manga['genre']=None
    try:
        manga['description']=driver.find_element(By.XPATH, "/html/body/div[1]/div[2]/div[3]/div[2]/table/tbody/tr/td[2]/div[1]/table/tbody/tr[1]/td/span").text.split('[Written')[0]
    except NoSuchElementException:
        manga['description']=None
    manga['serialization']=driver.find_element(By.XPATH, "//div[@class='spaceit_pad'][span[text()='Serialization:']]").text.split(': ')[1]
    driver.get(manga_dict['href']+'/stats') #truy cập vào đường dẫn chứa thông tin chi tiết số lượng người đang theo dõi manga
    manga['reading']=driver.find_element(By.XPATH, "//div[@class='spaceit_pad'][span[text()='Reading:']]").text.split(': ')[1]
    manga['completed']=driver.find_element(By.XPATH, "//div[@class='spaceit_pad'][span[text()='Completed:']]").text.split(': ')[1]
    manga['dropped']=driver.find_element(By.XPATH, "//div[@class='spaceit_pad'][span[text()='Dropped:']]").text.split(': ')[1]
    manga['plan_to_read']=driver.find_element(By.XPATH, "//div[@class='spaceit_pad'][span[text()='Plan to Read:']]").text.split(': ')[1]
    return manga


In [155]:
m=manga_info(manga_list[2])
m

{'name': 'Subete no Jinrui wo Hakai suru. Sorera wa Saisei Dekinai.',
 'ranked': 484,
 'score': '8.05',
 'popularity': '1026',
 'favorite': '346',
 'published': 'Nov 26, 2018',
 'genre': None,
 'description': 'The year is 1998. Hajime Kanou is a junior high student enthralled with the collectible card game Magic: The Gathering—dubbed "Magic" for short—playing with his classmates every day during breaks. His loud group often catches the attention of Emi Sawatari, the top student who berates Hajime\'s group for bringing such games to school. Hajime sees Emi as his self-proclaimed rival and challenges her to academic battles, which frequently result in his defeat.\n\nOne day, Hajime pays a visit to a card shop that specializes in Magic. Much to his surprise, not only does he find Emi inside, but he also discovers that she is the undisputed best Magic player in the store. This meeting sparks a slow but steady change in their relationship, as they grow to know each other better through vari

# Chuyển đổi data thu thập được sang dataFrame rồi lưu lại file csv

In [ ]:
data= []
for m in manga_list:
    manga_inf=manga_info(m)
    data.append(manga_inf)


In [162]:
df=pd.DataFrame.from_dict(data)
df

,name,ranked,score,popularity,favorite,published,genre,description,serialization,reading,completed,dropped,plan_to_read
0,Berserk,1,9.47,1,"122,974","Aug 25, 1989","Action, Adventure, Award Winning, Drama, Fanta...","Guts, a former mercenary now known as the ""Bla...",Young Animal,"364,280","87,307","10,263","154,118"
1,JoJo no Kimyou na Bouken Part 7: Steel Ball Run,2,9.30,26,"42,935","Jan 19, 2004","Action, Adventure, Mystery, Supernatural","In the American Old West, the world's greatest...",Ultra Jump,"34,922","162,358","2,141","50,652"
2,Vagabond,3,9.24,15,"40,221","Sep 3, 1998","Action, Adventure, Award Winning","In 16th-century Japan, Shinmen Takezou is a wi...",Morning,"108,564","81,383","5,054","145,679"
3,One Piece,4,9.22,3,"114,647","Jul 22, 1997","Action, Adventure, Fantasy","Gol D. Roger, a man referred to as the ""King o...",Shounen Jump (Weekly),"465,243","30,242","18,817","46,908"
4,Monster,5,9.15,29,"20,528","Dec 5, 1994","Award Winning, Drama, Mystery","Kenzou Tenma, a renowned Japanese neurosurgeon...",Big Comic Original,"33,413","94,357","2,754","97,297"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Nanairo Sekai,996,7.78,3246,39,"Nov 28, 2005","Boys Love, Romance",A series of one-shots that ran in Monthly Melo...,Melody,189,"4,503",60,"1,288"
996,Shinobi Life,997,7.78,669,433,"Feb 6, 2006","Action, Comedy, Drama, Fantasy, Romance, Super...","Energetic and strong-willed, Beni doesn't care...",Princess,"5,270","8,468","1,522","8,741"
997,Asterisk,998,7.78,6977,53,"Mar 20, 2006","Action, Boys Love, Fantasy, Supernatural","Frau and Kio are angels, and it's their duty t...",Ichiraci,459,"1,105",106,840
998,Shanaou Yoshitsune,999,7.78,10658,23,"Nov 6, 2000",Award Winning,Shanaou Yoshitsune is an alternative story to ...,Shounen Magazine (Monthly),368,444,51,587


In [163]:
df.to_csv('data.csv',index=False)

In [164]:
d=pd.read_csv('data.csv')
d

,name,ranked,score,popularity,favorite,published,genre,description,serialization,reading,completed,dropped,plan_to_read
0,Berserk,1,9.47,1,"122,974","Aug 25, 1989","Action, Adventure, Award Winning, Drama, Fanta...","Guts, a former mercenary now known as the ""Bla...",Young Animal,"364,280","87,307","10,263","154,118"
1,JoJo no Kimyou na Bouken Part 7: Steel Ball Run,2,9.30,26,"42,935","Jan 19, 2004","Action, Adventure, Mystery, Supernatural","In the American Old West, the world's greatest...",Ultra Jump,"34,922","162,358","2,141","50,652"
2,Vagabond,3,9.24,15,"40,221","Sep 3, 1998","Action, Adventure, Award Winning","In 16th-century Japan, Shinmen Takezou is a wi...",Morning,"108,564","81,383","5,054","145,679"
3,One Piece,4,9.22,3,"114,647","Jul 22, 1997","Action, Adventure, Fantasy","Gol D. Roger, a man referred to as the ""King o...",Shounen Jump (Weekly),"465,243","30,242","18,817","46,908"
4,Monster,5,9.15,29,"20,528","Dec 5, 1994","Award Winning, Drama, Mystery","Kenzou Tenma, a renowned Japanese neurosurgeon...",Big Comic Original,"33,413","94,357","2,754","97,297"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Nanairo Sekai,996,7.78,3246,39,"Nov 28, 2005","Boys Love, Romance",A series of one-shots that ran in Monthly Melo...,Melody,189,"4,503",60,"1,288"
996,Shinobi Life,997,7.78,669,433,"Feb 6, 2006","Action, Comedy, Drama, Fantasy, Romance, Super...","Energetic and strong-willed, Beni doesn't care...",Princess,"5,270","8,468","1,522","8,741"
997,Asterisk,998,7.78,6977,53,"Mar 20, 2006","Action, Boys Love, Fantasy, Supernatural","Frau and Kio are angels, and it's their duty t...",Ichiraci,459,"1,105",106,840
998,Shanaou Yoshitsune,999,7.78,10658,23,"Nov 6, 2000",Award Winning,Shanaou Yoshitsune is an alternative story to ...,Shounen Magazine (Monthly),368,444,51,587
